## TEST - BWT

### Load Image

In [ ]:
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from bwt_encryption.bwt2dv2 import bwt2dv2
from bwt_encryption.ibwt2dv2 import ibwt2dv2

im = np.array(Image.open('./images/lenna_gray.png').convert('L'))

# Adds one more pixel at coordinate [0,0]
modified_im = im.copy()
modified_im[0,0] = (modified_im[0,0] + 1) % 255
plt.figure(figsize=(3,3))
plt.imshow(im, cmap='gray',)
print(f"Image size: {im.shape}")

### Encrypt and Decrypt

In [ ]:
keys = [0.967198510247, 0.294368563711, 0.867612365755, 0.545395617916]

encrypt_1 = ibwt2dv2(im=im, keys=keys)

decrypt_1 = bwt2dv2(im=encrypt_1, keys=keys)

encrypt_2 = ibwt2dv2(im=modified_im, keys=keys)

decrypt_2 = bwt2dv2(im=encrypt_2, keys=keys)

### Preview

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(10, 6))
images = [(encrypt_1, 'Encrypted - Original'), 
          (encrypt_2, 'Encrypted - Modified'), 
          (decrypt_1, 'Decrypted - Original'), 
          (decrypt_2, 'Decrypted - Modified')]

for ax, (img, title) in zip(axs, images):
    ax.imshow(img, cmap='gray')
    ax.set_title(title, fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

### UACI and NPCR Test

In [ ]:
from security_tests.differential_attacks import uaci_npcr_metrics
uaci_npcr_metrics(encrypt_1, encrypt_2)

### Histogram Analysis

In [ ]:
from security_tests.statistic import analyze_encryption_statistics
analyze_encryption_statistics(im, encrypt_1)

### MSE and SSIM Test

In [ ]:
from security_tests.key_sensitivity import evaluate_key_sensitivity
evaluate_key_sensitivity(
    original_image=im, 
    encrypted_image=encrypt_1, 
    original_key=keys, 
    decrypt_function=bwt2dv2, 
    num_variations=100
)

### NIST Test

Converte imagem criptografada para binário e executa a bateria de testes.

In [ ]:
from security_tests.nist_test.convert_image_to_bits import image_to_bits
convert_image =  image_to_bits(encrypt_1, './security_tests/nist_test/bit_sequence.txt')
%run ./security_tests/nist_test/nist_main.py

### Speed Test

In [ ]:
from security_tests.speed_test import SpeedTester

# Note: If you only wish to test specific images, like avoiding the larger ones (2048 or 4096), 
# you can leave those lines commented, as shown above.
im = {
    256: np.array(Image.open('./images/cameraman_gray.png').convert('L')),
    512: np.array(Image.open('./images/lenna_gray.png').convert('L')),
    1024: np.array(Image.open('./images/airplaneU2.png').convert('L')),
    2048: np.array(Image.open('./images/the_last_of_us.jpg').convert('L')),
    4096: np.array(Image.open('./images/sun_gray.png').convert('L')),
}

speed_tester = SpeedTester(images=im, encrypt_function=ibwt2dv2, keys=keys, num_rounds=2)
result =speed_tester.run_tests()
speed_tester.plot_results(average_times=result)

## Mean Metrics using a Dataset

### UACI and NPCR Mean

In [ ]:

from security_tests.differential_attacks import process_images
directory = './dataset/512/'
process_images(directory=directory, 
               keys=keys, 
               rounds=2, 
               encrypt_function=ibwt2dv2, 
               limit=None)